In [11]:
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG
from metric_learn import Covariance

from IPython.display import Image, display
from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn.cluster import KMeans

In [12]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, material = read_VSP_group(pappas_only=False)

D = count_matrix(groups_VSP, raw_feature.shape[0])
clusters_VSP = spectralClusteringDBSCAN(D, threshold=0,eps=0.001)

print(raw_feature.shape)
print(label_material.shape)

(5245, 82)
(5245,)


In [13]:
label_visiprog = get_VSP_label(groups_VSP, raw_feature.shape[0])
label_VSP_clusters = get_VSP_label(clusters_VSP, raw_feature.shape[0])

In [14]:
model = Covariance()
model.fit(raw_feature)
X_STSIMM = model.transform(raw_feature)
# print(X_STSIMM)

In [16]:
accuracy_STSIMM = []
accuracy_mat = []
accuracy_vsp_original = []
accuracy_vsp = []

N_material = 61
N_material_train = 50

label_visiprog = label_VSP_clusters

for fold in range(10):
    print("Trial {}".format(fold))

    # randomly select N_material_train material for training the metric
    # and test on the rest
    train_select = np.zeros_like(label_material).astype(bool)

    idx_train = np.array([])
    for i in range(N_material_train):
        train_class = np.random.randint(0, N_material)
        train_select |= (label_material == train_class)

    idx_train = np.where(train_select == True)[0]
    idx_test = np.where(train_select == False)[0]

    idx_train = idx_train.astype(int)
    idx_test = idx_test.astype(int)

    print("Training shape {} and test shape {}".format(len(idx_train), len(idx_test)))

    X_train = raw_feature[idx_train,:]
    X_STSIMM_train = X_STSIMM[idx_train,:]
    Y_VSP_train = label_visiprog[idx_train]
    Y_mat_train = label_material[idx_train]

    X_test = raw_feature[idx_test,:]
    X_STSIMM_test = X_STSIMM[idx_test,:]
    Y_VSP_test = label_visiprog[idx_test]
    Y_mat_test = label_material[idx_test]

    res_STSIMM = leave_one_sample_out(X_STSIMM_test, Y_mat_test)
    accuracy_STSIMM.append(res_STSIMM['accuracy'])

    model = LFDA()
    try:
        model.fit(X_train, Y_mat_train)
        X_mat_test = model.transform(X_test)
        res = leave_one_sample_out(X_mat_test, Y_mat_test)
        accuracy_mat.append(res['accuracy'])
    except Exception as e:
        acuracy_mat.append(np.nan)

    model_vsp_original = LFDA()
    try:
        # remove the unlabeled data
        idx_outlier = np.where(Y_VSP_train == 0)[0]
        X_train_without_outlier = X_train[~idx_outlier,:]
        Y_VSP_train_without_outlier = Y_VSP_train[~idx_outlier]
        
        model_vsp_original.fit(X_train_without_outlier, Y_VSP_train_without_outlier)
        X_vsp_original_test = model_vsp_original.transform(X_test)
        res_vsp_original = leave_one_sample_out(X_vsp_original_test, Y_mat_test)
        accuracy_vsp_original.append(res_vsp_original['accuracy'])
    except Exception as e:
        accuracy_vsp_original.append(np.nan)

    model_vsp = LFDA_VISIPROG()
    try:
        model_vsp.fit(X_train, Y_VSP_train)
        X_vsp_test = model_vsp.transform(X_test)
        res_vsp = leave_one_sample_out(X_vsp_test, Y_mat_test)
        accuracy_vsp.append(res_vsp['accuracy'])
    except Exception as e:
        accuracy_vsp.append(np.nan)

Trial 0
Training shape 2546 and test shape 2699
Trial 1
Training shape 2730 and test shape 2515
Trial 2
Training shape 3046 and test shape 2199
Trial 3
Training shape 3101 and test shape 2144
Trial 4
Training shape 2886 and test shape 2359
Trial 5
Training shape 3174 and test shape 2071
Trial 6
Training shape 2966 and test shape 2279
Trial 7
Training shape 3067 and test shape 2178
Trial 8
Training shape 2691 and test shape 2554
Trial 9
Training shape 3063 and test shape 2182


In [17]:
df = pd.DataFrame({'visiprog':accuracy_vsp, 'material':accuracy_mat, 'STSIMM':accuracy_STSIMM, \
       'visiprog_original':accuracy_vsp_original}, columns=['STSIMM','material','visiprog_original','visiprog'])

df.loc['average'] = df.mean()

df.to_csv('thrasos/across_50material_nearest_neighbor.csv')

display(df)
# df.to_csv('acr')
# display(df.mean())

,STSIMM,material,visiprog_original,visiprog
0,0.896258,0.990737,0.974435,0.986662
1,0.867197,0.993638,0.958250,0.990457
2,0.920418,0.989995,0.972715,0.989541
3,0.926306,0.989272,0.980877,0.990672
4,0.903773,0.992370,0.976261,0.992794
5,0.900531,0.993723,0.984066,0.994689
6,0.911803,0.989908,0.977622,0.992979
7,0.910927,0.991276,0.983930,0.990817
8,0.917776,0.994127,0.981597,0.994127
9,0.916590,0.993126,0.978460,0.994500


In [7]:
print(model.transformer())

[[  5.15406457e+16   1.77802384e+02  -2.35126850e+00 ...,   2.04218482e-01
    7.12850006e-01  -2.78102062e+00]
 [ -1.46337803e+17  -3.05401739e+01   1.70971345e-01 ...,  -9.67410184e-01
   -1.45023383e+00  -6.15406957e-01]
 [ -3.28236673e+16   3.44866491e+00   6.83016962e-01 ...,  -1.09290320e+00
   -7.89008999e-01  -1.71855305e+00]
 ..., 
 [ -5.49120769e+16   2.25034254e+01   7.46105478e-02 ...,   1.46315936e-01
   -9.10106855e-01   1.89370358e-01]
 [  3.29349403e+16  -5.61026935e+00  -1.08319468e-01 ...,  -9.60830277e-02
   -1.40494185e-01   2.18494058e-01]
 [ -1.26825085e+16   8.60702260e+00  -9.10717574e-02 ...,   3.94526462e-02
   -1.20452139e-01   3.51024985e-01]]
